In [151]:
BLACK = '\033[30m'
RED = '\033[31m'
GREEN = '\033[32m'
YELLOW = '\033[33m'
BLUE = '\033[34m'
MAGENTA = '\033[35m'
CYAN = '\033[36m'
WHITE = '\033[37m'
ORANGE = '\033[93m'
# Bright Colors
BRIGHT_BLACK = '\033[90m'
BRIGHT_RED = '\033[91m'
BRIGHT_GREEN = '\033[92m'
BRIGHT_YELLOW = '\033[93m'
BRIGHT_BLUE = '\033[94m'
BRIGHT_MAGENTA = '\033[95m'
BRIGHT_CYAN = '\033[96m'
BRIGHT_WHITE = '\033[97m'

# Reset
RESET = '\033[0m'

class Contract:
    def __init__(self, address, upgr_functions, bytecode_hash, kind, decompilation_status, imp_get_function = ""):
        self.upgr_functions = upgr_functions
        self.address = address
        self.kind = kind
        self.decompilation_status = decompilation_status
        self.bytecode_hash = bytecode_hash
        self.imp_get_function = imp_get_function
        
    def print_instance_attributes(self):
        GREEN = '\033[92m'
        RESET = '\033[0m'
        print(f"\t    {ORANGE}CONTRACT ADDRESS:{RESET} {BLACK}{self.address}{RESET}")
        print(f"\t\t{CYAN}KIND:{RESET} {BLACK}{self.kind}{RESET}")
        print(f"\t\t{CYAN}DECOMPILATION STATUS:{RESET} {BLACK}{self.decompilation_status}{RESET}")
        print(f"\t\t{CYAN}BYTECODE HASH:{RESET} {BLACK}{self.bytecode_hash}{RESET}")
        print(f"\t\t{CYAN}IMPLEMENTATION GET FUNCTION SELECTOR:{RESET} {BLACK}{self.imp_get_function}{RESET}")
        if len(self.upgr_functions) > 0:
            for idx, upg_func in enumerate(self.upgr_functions):
                print(f"\t\t{CYAN}UPGRADE FUNCTION SIGNATURE #{idx}:{RESET} {BLACK}{upg_func}{RESET}")
        print("\t\t------------")

class RelevantDelegatecall:

    def __init__(self):
        self.proxy_func_sig = ""
        self.proxy_func_loc = ""
        self.is_upc = False
        self.upg_ref_design = ""
        self.upg_proxy_pattern = ""
        self.impact_variables = ""
        self.delegatecall_line = ""
        self.delegatecall_loc = ""
        self.contracts = []
        self.flag = ""
        self.metadata = ""

    def add_contract(self, contract):
        if isinstance(contract, Contract):
            self.contracts.append(contract)
        c = 0
        for con in self.contracts:
            if con.decompilation_status.find("FAILURE")>=0:
                c+=1
        # if all contracts for this relevant delegatecall failed during decompilation, set the is_upc flag to decompilation failure
        if c == len(self.contracts):
            self.is_upc = "FAILURE:DECOMPILER:{}".format(self.contracts[-1].kind)
        
    def print_instance_attributes(self):
        GREEN = '\033[92m'
        RESET = '\033[0m'
        RED = '\033[91m'
        print(f"\t{MAGENTA}DELEGATECALL LINE:{RESET} {BLACK}{self.delegatecall_line}{RESET}")
        print(f"\t{MAGENTA}DELEGATECALL LOC:{RESET} {BLACK}{self.delegatecall_loc}{RESET}")
        print(f"\t{MAGENTA}PROXY FUNCTION SIGNATURE:{RESET} {BLACK}{self.proxy_func_sig}{RESET}")
        print(f"\t{MAGENTA}PROXY FUNCTION LOC:{RESET} {BLACK}{self.proxy_func_loc}{RESET}")
        print(f"\t{MAGENTA}IS UPC:{RESET} {BLACK}{self.is_upc}{RESET}")
        print(f"\t{MAGENTA}UPGRADEABILITY REFERENCE DESIGN:{RESET} {BLACK}{self.upg_ref_design}{RESET}")
        # print(f"\t{MAGENTA}UPGRADEABILITY PROXY PATTERN:{RESET} {BLACK}{self.upg_proxy_pattern}{RESET}")
        print(f"\t{MAGENTA}IMPACT VARIABLES SLOTS:{RESET} {BLACK}{self.impact_variables}{RESET}")
        for contract in self.contracts:
            contract.print_instance_attributes()


In [154]:
class Proxy:
    def __init__(self, address, impl_addresses, is_proxy_static_detector=False, is_proxy_dynamic_detector= False):
        """
        Constructor for the Proxy class, initializing attributes related to proxy detection results.
        """
        self.relevant_delegatecalls = list()
        self.is_upc = False
        self.addr = address
        self.impl_addresses = list(impl_addresses)
        self.is_proxy_static_detector = is_proxy_static_detector
        self.is_proxy_dynamic_detector = is_proxy_dynamic_detector
        self.decompilation_status = ""
        self.bytecode_hash = ""
        self.is_variant = False

    def determine_proxy_label(self):
        # if for one of the relevant delegate contracts we found that proxy is upc we set the is_upc flag to True
        for rev_del in self.relevant_delegatecalls:
            if rev_del.is_upc==True:
                self.is_upc = True
                break
            
        # if the is_upc flag is not true. then we need to check wheather there all relevant delegatecalls are properly evaluated or not. if so, is_upc label will false; otherwise, decompiler failure.
        c = 0
        if self.is_upc != True:
            for rev_del in self.relevant_delegatecalls:
                if rev_del.is_upc==False:
                    c+=1
            # if all relevant delegatecalls are non-upcs, then we can set the is_upc flag to false
            if c == len(self.relevant_delegatecalls):
                self.is_upc = False
            # otherwise this indicates we could not evaluate at least one relevant delegatecall (e.g., due to decompilation failure of all its contracts) properly and set the is_upc flag
            else:
                for rev_del in self.relevant_delegatecalls:
                    if rev_del.is_upc!=False:
                        self.is_upc = rev_del.is_upc
                        break
    
    def add_delegatecall(self, delegatecall):
        if isinstance(delegatecall, RelevantDelegatecall):
            self.relevant_delegatecalls.append(delegatecall)
        self.determine_proxy_label()
                
    def is_upgradeability_proxy(self):
        GREEN = '\033[92m'
        RED = '\033[91m'
        RESET = '\033[0m'
        print(f"{BLUE}PROXY:{RESET} {BLACK}{self.addr}{RESET}")
        print(f"    {RED}IS PROXY STATIC DETECTOR:{RESET} {BLACK}{self.is_proxy_static_detector}{RESET}")
        print(f"    {RED}IS PROXY DYNAMIC DETECTOR:{RESET} {BLACK}{self.is_proxy_dynamic_detector}{RESET}")
        print(f"    {RED}IS UPC:{RESET} {BLACK}{self.is_upc}{RESET}")
        print(f"    {RED}IS VARIANT:{RESET} {BLACK}{self.is_variant}{RESET}")
        print(f"    {RED}DECOMPILATION STATUS :{RESET} {BLACK}{self.decompilation_status}{RESET}")
        print(f"    {RED}BYTECODE HASH:{RESET} {BLACK}{self.bytecode_hash}{RESET}")
        print(f"    {RED}IMPLEMENTATION CONTRACTS COUNT:{RESET} {BLACK}{len(self.impl_addresses)}{RESET}")
        for idx, delegatecall in enumerate(self.relevant_delegatecalls):
            print(f"    {RED}DELEGATECALL #{idx}:{RESET}")
            delegatecall.print_instance_attributes()
        print(f"{BLUE}===================================================================={RESET}")